In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# input

parsed_finemap_f <- 'FINEMAP_output.tsv'
traits_f <- '../common/canonical_trait_names.txt'




In [3]:
source('finemap_functions.R')

In [4]:
traits <- fread(traits_f) %>%
filter(annotation != 'Fasting_glucose') %>%
pull(annotation)

df <- fread(parsed_finemap_f) %>%
filter(trait %in% traits)

res <- list()
for(t in traits){
    res[[ t ]] <- df %>% filter(trait == t) %>% select(-trait)
}


#### compute statistics for the summary figure

In [5]:
n_regions <- sum( sapply( res, get_n_regions ) )
n_signals <- sum( sapply( res, get_n_signals ) )
n_signal_per_region <- table( unlist( lapply( res, get_n_signal_per_region ) ) )
n_snps_per_signal <- table( unlist( lapply( res, get_n_snps_per_signal) ) )
n_signal_per_region_max <- setNames(lapply(res, get_n_signal_per_region_max), names(res))


#### Number of distinct signals per region

In [6]:
get_summary_number_of_distinct_signals_per_region(n_signal_per_region, n_signal_per_region_max, length(res))

Distinct association signals. A single signal at 33 regions, and two to 40 signals at 5330 regions across 35 traits.


n_signals,1,2,3,4,5,6+
n_regions,33,276,963,1256,1033,1802


#### Number of variants in the credible set with >= 99%% posterior probability

In [7]:
get_summary_number_of_variants_in_credible_set(n_snps_per_signal, length(res))


Number of variants in the credible set with >= 99% posterior probability. In total of 2547 signals were mapped to a single variant in the credible set across 35 traits.


n_variants,1,2-5,6-10,11-20,21-50,51+
n_signals,2547,3862,2510,3199,5578,10157


In [9]:
pdf(file.path('figs', 'fig3A.pdf'), width = 12, height = 3)
plot_FINEMAP_summary(n_regions, n_signals, n_signal_per_region, n_snps_per_signal, get_colors6())
dev.off()


png 
  2